In [1]:
%matplotlib notebook

In [2]:
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score

In [3]:
x,y = make_regression(n_samples=100,n_features=1,n_informative=1,n_targets=1,noise=20,random_state=13)

In [4]:
plt.scatter(x,y)

<IPython.core.display.Javascript object>

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [6]:
# lets' apply ols
from sklearn.linear_model import LinearRegression


In [7]:
lr = LinearRegression()


In [8]:
lr.fit(x_train,y_train)
print(lr.coef_)
print(lr.intercept_)

[28.12597332]
-2.2710144261783825


In [9]:
y_pred = lr.predict(x_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.6345158782661012

In [10]:
class GDRegressor:

    def __init__(self,learning_rate,epochs):
        self.m = 100
        self.b = -120
        self.lr = learning_rate
        self.epochs = epochs

    def fit(self, x, y):
        # calculate the b using GD
        for i in range(self.epochs):
            loss_slope_b = -2 * np.sum(y - self.m * x.ravel() - self.b)
            loss_slope_m = -2 * np.sum((y - self.m * x.ravel() - self.b)*x.ravel())
            self.b = self.b - (self.lr * loss_slope_b)
            self.m = self.m - (self.lr * loss_slope_m)
            
        print(self.m,self.b)

    def predict(self, x):
        return self.m * x + self.b


In [11]:
gd = GDRegressor(0.001,50)

In [12]:
gd.fit(x_train,y_train)

28.159367347119066 -2.3004574196824854


In [13]:
y_pred = gd.predict(x_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.6343842836315579

In [19]:
b = 600
m = -127.82
lr = 0.001
all_b = []
all_m = []
all_cost = []

epochs = 30

for i in range(epochs):
    slope_b = 0
    slope_m = 0
    cost = 0
    for j in range(x.shape[0]):
        slope_b = slope_b - 2*(y[j] - (m * x[j]) -  b)
        slope_m = slope_m - 2*(y[j] - (m * x[j]) -  b)*x[j]
        cost = cost + (y[j] - m * x[j] -b) ** 2

    b = b - (lr * slope_b)
    m = m - (lr * slope_m)
    all_b.append(b)
    all_m.append(m)
    all_cost.append(cost)

In [20]:
from matplotlib.animation import FuncAnimation
fig , ax = plt.subplots(figsize=(9,5))
x_i = np.arange(-3,3,0.1)
y_i = x_i*(-27)-150
ax.scatter(x, y)
line, = ax.plot(x_i, x_i*50-4, 'r-', linewidth=2)
def update(i):
    label = 'epoch {0}'.format(i+1)
    line.set_ydata(x_i * all_m[i] + all_b[i])
    ax.set_xlabel(label)
anim = FuncAnimation(fig, update, repeat=True, frames=epochs, interval=500)
plt.show()
from matplotlib.animation import PillowWriter
anim.save('animation.gif', writer=PillowWriter(fps=2))
anim

<IPython.core.display.Javascript object>

In [21]:
import plotly.express as px
import plotly.graph_objects as go

# Define meshgrid for m and b
m_arr = np.linspace(min(np.array(all_m).ravel())-10, max(np.array(all_m).ravel())+10, 50)
b_arr = np.linspace(min(np.array(all_b).ravel())-10, max(np.array(all_b).ravel())+10, 50)
M, B = np.meshgrid(m_arr, b_arr)

# Calculate cost for each (m, b) pair
z_arr = np.zeros_like(M)
for i in range(M.shape[0]):
	for j in range(M.shape[1]):
		y_pred_mesh = M[i, j] * x.ravel() + B[i, j]
		z_arr[i, j] = np.sum((y - y_pred_mesh) ** 2)

fig = px.scatter_3d(x=np.array(all_m).ravel(), y=np.array(all_b).ravel(), z=np.array(all_cost).ravel()*100)

fig.add_trace(go.Surface(x = m_arr, y = b_arr, z =z_arr*100 ))

fig.show()

fig.write_html("cost_function2.html")

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter(x=np.array(all_m).ravel(), y=np.array(all_b).ravel(), name='High 2014',
                         line=dict(color='#fff', width=4)))

fig.add_trace(go.Contour(z=z_arr,x=m_arr,y=b_arr))
fig.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


fig,ax=plt.subplots(1,1)
plt.figure(figsize=(18,4))
cp = ax.contourf(m_arr, b_arr, z_arr)
ax.plot(np.array(all_m).ravel(),np.array(all_b).ravel(),color='white')
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Filled Contours Plot')
ax.set_xlabel('m')
ax.set_ylabel('b')
plt.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:

from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation

# intercept graph
num_epochs = list(range(0,30))
fig = plt.figure(figsize=(9,5)) 
axis = plt.axes(xlim =(-150, 150), ylim =(-150, 150)) 

axis.contourf(m_arr, b_arr, z_arr)

line, = axis.plot([], [], lw = 2,color='white') 
  
xdata, ydata = [], [] 
  
# animation function 
def animate(i): 
    label = 'epoch {0}'.format(i + 1)
    xdata.append(all_m[i]) 
    ydata.append(all_b[i]) 
    line.set_data(xdata, ydata) 
    axis.set_xlabel(label)
      
    return line, 
  
# calling the animation function     
anim = animation.FuncAnimation(fig, animate, frames = 30,repeat=False, interval = 500)

f = r"animation8.gif" 
writergif = animation.PillowWriter(fps=2) 
anim.save(f, writer=writergif)

<IPython.core.display.Javascript object>